In [ ]:
# !pip3 install chardet -i https://mirrors.aliyun.com/pypi/simple/

In [ ]:
import glob
import os
from tqdm import tqdm
import numpy as np
import shutil

import IPython
def play_audio(data, rate):
    IPython.display.display(IPython.display.Audio(data=data,rate=rate))

In [ ]:
sr = 16000
result_base_dir = "./audio/"
personalities = ["Openness", "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
batch_dirs = glob.glob(result_base_dir + "*/")
batch_dirs.sort()

# MUSHRA

In [ ]:
save = True
testname = "Personality Labeling Survey"
pngfile = "img/alignment.png"
# text = '<b>Five Personalities we cover in this survey. </b><br><ul><li> <b>Openness</b>: intellectual, imaginative, independent-minded; opposite term is Closeness. <li> <b>Conscientiousness</b>: orderly, responsible, dependable; opposite term is Lack of direction. <li> <b>Extraversion</b>: talkative, assertive, energetic; opposite term is Introversion. <li> <b>Agreeableness</b>: good-natured, cooperative, trustful; opposite term is Antagonism. <li> <b>Neuroticism</b>: emotional instability, irritability, anxiety, self-doubt, depression; opposite term is Emotion stability. </ul> '
text = 'Please move the slider to input the alignment score between a speaker and a personality given 90-seconds dialog between two speakers. In Stereo Sound, you can hear different speakers from different ears. <b>Please double check the personality label and the speaker when you score.</b> We cover the following five personalities in this survey. <br><br> <b>Openness</b>: intellectual, imaginative, independent-minded; opposite term is <i>Closeness</i>. <br> <b>Conscientiousness</b>: orderly, responsible, dependable; opposite term is <i>Lack of direction</i>. <br> <b>Extraversion</b>: talkative, assertive, energetic; opposite term is <i>Introversion</i>. <br> <b>Agreeableness</b>: good-natured, cooperative, trustful; opposite term is <i>Antagonism</i>. <br> <b>Neuroticism</b>: emotional instability, irritability, anxiety, self-doubt, depression; opposite term is <i>Emotion stability</i>.  '
def init():
    whole_text = f"""var TestConfig = {{
  "TestName": "{testname}",
  "RateScalePng": "{pngfile}",
  "RateScaleBgPng": "img/scale_abs_background.png",
  "RateMinValue": 0,
  "RateMaxValue": 100,
  "RateDefaultValue":0,
  "ShowFileIDs": false,
  "ShowResults": false,
  "LoopByDefault": false,
  "EnableABLoop": true,
  "EnableOnlineSubmission": false,
  "BeaqleServiceURL": "/web_service/beaqleJS_Service.php",
  "SupervisorContact": "shoinoue@link.cuhk.edu.cn",
  "RandomizeTestOrder": true,
  "MaxTestsPerRun": 10000,
  "RequireMaxRating": false,
  "AudioRoot": "",
  "Testsets":
["""
    return whole_text

for b in range(len(batch_dirs)):
    batchdir = batch_dirs[b]
    whole_text = init()
    paths = glob.glob(batchdir + "*/")
    paths.sort()
    for path in paths:
        fn = os.path.basename(path[:-1])
        left = path + "0.wav"
        right = path + "1.wav"
        stereo = path + "stereo.wav"
        mono = path + "mono.wav"
        starter = f"""
      {{
        "Name": "{fn}",
        "TestID": "{b}___{fn}",
        "Text": "{text}",
        "Files": {{\n"""
        modelpaths = """"""
        modelpaths += f'      "Stereo": "{stereo}",\n'
        modelpaths += f'      "Mono": "{mono}",\n'
        modelpaths += f'      "Left": "{left}",\n'
        modelpaths += f'      "Right": "{right}",\n'
        for mode in ["Left Speaker", "Right Speaker"]:
            for ps in personalities:
                added = f'      "{ps}<br>({mode})": "./audio/sample.wav",\n'
                modelpaths += added
        ##########################
        modelpaths += f'    }}\n'
        modelpaths += f'  }},'
        whole_text += starter+modelpaths
    whole_text += "\n]\n}}"
    # print(whole_text)

    savefile = f"./config/human_label_{b}.js"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(whole_text[:-1])
        f.close()

# HTML

In [ ]:
beg = """<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
    <meta http-equiv="content-type" content="application/xhtml+xml; charset=UTF-8" />

    <title>BeaqleJS</title>
    
"""
end = """
    
    <link type="text/css" href="css/smoothness/jquery-ui-1.8.18.custom.css" rel="Stylesheet" />
    <link type="text/css" href="css/styles.css" rel="Stylesheet" />

    <script src="js/jquery.js" type="text/javascript"></script>
    <script src="js/jquery-ui.custom.min.js" type="text/javascript" ></script>
    <script src="js/beaqle.js" type="text/javascript"></script>


    <script type="text/javascript">
    /* <![CDATA[ */
        var testHandle;
        window.onload=function() {

            // Uncomment one of the following lines to choose the desired test class

            testHandle = new MushraLabel(TestConfig);    // <-- MUSHRA test class

            $('#Footer').prepend(testHandle.browserFeatureString() + '<br/>');
        };
    /* ]]> */
    </script>
   
</head>

<body>

<div id="Wrapper">

    <div id="TestTitle" class="commonBox">
        <!-- overall title at the top 
             DO NOT EDIT! This box is automatically filled with content!
        -->
        Loading...
    </div>
    
    <!-- is displayed during load -->
    <div id="LoadOverlay" class="commonBox">
        <img src="img/ajax-loader.gif"/> &nbsp;&nbsp;&nbsp; Loading...
    </div>

    <!-- introduction before the test starts -->
    <div id="TestIntroduction" class="commonBox">
        
        <!-- edit here if you want to change the welcome message  and instructions -->

        <div style="text-align:left">
            Thanks for your interest in participating in the listening test. This survey will roughly take 60 minutes. You will listen to 19 dialogues. The second experiment is to determine the personalities given dialogues with two people. After you listen to dialog, you are required to answer whether the speaker is aligned/opposed to each personality. 
            <br />
            <br />
             Please listen to the audio clips carefully and follow the instruction below:
            <ul>
                <li>Each speaker is separated from another by listening channels (You will hear different speakers from different ears). So, be sure to wear earphones/headphones correctly. 
                <li>You can have two or more aligned/opposed answers for each speaker (e.g. If you think a speaker is characterized by combination of “openness” and “agreeableness”, you can answer “aligned” for both personalities)
                <li>For each audio file, you must give answers for two speakers.
                <li>Move the slider to give alignment scores for each personality and speaker.</li>
            </ul>
            There are ways for checking your attention through the survey. If a certain number of the scores are rated unreasonably, you will not be paid. So, listen to the audio carefully and do not fill out the survey randomly. Please <b> wear </b> headphones and be in a quiet room.
        </div>
        <br />
        <br />
                        
        <!-- this button starts the first test -->
        <button id="BtnStartTest" onclick="testHandle.startTests();">Start!</button>	
        
<!--         <br /><br />
        <div style="text-align:left">
            <b> Sample Demo Page: </b>
        </div><br /> -->
        
        <!-- is displayed during load -->
<!--         <div id="demonat">
            <img src="img/nat.jpg" alt="demonat"  width="500">
        </div> -->
    </div>

    <!-- final text after all tests have passed -->
    <div id="TestEnd" class="commonBox">

        <!-- edit here if you want to change the thank you message -->
        Thank you very much!
        <br />
        <br />
        
        <!-- show some informations before submitting the results -->
        <div id="SubmitBox">
            <div id="SubmitError" class="error">
                We are sorry, but the following error occured during your submission:
                <p id="ErrorCode"></p>
            </div>
            <p class="submitOnline">
                <!-- edit here if you want to change the information about online submission, e.g. add privacy policy information -->
                Please submit the results to our server. Entering a name or email address is not mandatory
                but it would help us to contact you if we have further questions. You can also leave a comment.
            </p>
            <p class="submitDownload">
                <!-- edit here if you want to change the information about online submission, e.g. add privacy policy information -->
                Please download and save the results. Entering a name or email address is not mandatory 
                but it would help us to contact you if we have further questions. You can also leave a comment.
            </p>
            <table border=0 style="margin-left: auto;margin-right: auto;">
            <tr><td>Name:</td><td><input id="UserName" type="text" size="40" maxlength="40" /></td></tr>
            <tr><td>Email:</td><td><input id="UserEMail" type="text" size="40" maxlength="40" /></td></tr>
            <tr><td></td><td><textarea id="UserComment" cols="40" rows="4">Your comment...</textarea></td></tr>
            </table>
            <br />
            <button id="BtnSubmitData" class="submitOnline" onclick="testHandle.SubmitTestResults();">Submit</button>
            <button id="BtnDownloadData" class="submitDownload" onclick="testHandle.DownloadTestResults();">Download</button>
            <p class="submitEmail error">
                <!-- edit here if you want to change the information about email submission, e.g. add privacy policy information -->
                Results should be send to <span class="supervisorEmail"></span>
            </p>
        </div>
        
        <!-- the test results are displayed in this box -->
        <div id="ResultsBox">
        </div>
    </div>

    <div id="TableContainer" class="commonBox">
        <!-- container for the test items and sliders 
             DO NOT EDIT! This box is automatically filled with content!
        -->    

        <!-- heading for the single test -->
        <div id="TestHeading">
        </div>
    
    </div>

    <!-- controls to go to next/previous test -->
    <div id="TestControls" class="commonBox">
        <button id="BtnNextTest">Next Test</>
        <button id="BtnPrevTest">Previous Test</>
    </div>

    <!-- playback control -->
    <div id="PlayerControls" class="commonBox">

        <div id="duration">
            <span>00:00</span>
        </div>
        
        <div id="TimeLine">
            <div id="ProgressBar"></div>
            <div id="ABRange"></div>
        </div>
        
        <div>
            <input id="ChkLoopAudio" type="checkbox" value="Loop" /><label for="ChkLoopAudio">Loop</label>
            <input id="ChkAutoReturn" type="checkbox" value="Auto Return" /><label for="ChkAutoReturn">Auto Return</label>
        </div>
        
        <div id="Volume">
            Volume
            <div id="VolumeSlider"></div>
        </div>
        
        <br style="clear:both"/>
    
    </div>

    <!-- Footer -->
    <div id="Footer">
        This listening test has been created with <a href="https://github.com/HSU-ANT/beaqlejs" target="_blank">BeaqleJS</a> <a href="https://github.com/HSU-ANT/beaqlejs/tree/master" target="_blank">master</a>.
    </div>
</div>

</body>

</html>"""

b = 0
for b in range(len(batch_dirs)):
    add = f'        <script src="config/human_label_{b}.js" type="text/javascript"></script>'
    text = beg + add + end
    savefile = f"./human_label_{b}.html"
    try:
        os.remove(savefile)
    except FileNotFoundError:
        pass
    if save:
        f = open(savefile, "a")
        f.write(text)
        f.close()

In [ ]:
for b in range(len(batch_dirs)):
    text = f"""        <tr>
          <th scope="row">2-{b+1}</th>
          <td><i>Group {b+1}</i></td>
          <td>Survey to Collect Personality Labels Given Audio for Participant {b+1}</td>
          <td>about 60 minutes</td>
          <td>
            <a href="./human_label_{b}.html">Enter Test</a>
          </td>
        </tr>"""
    print(text)